<a href="https://colab.research.google.com/github/pubayankapat/Audio-emotion-detector/blob/main/music_emotion_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Pubayan Kapat,
Class roll no: 24,
Course: MCA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load data**

In [ ]:
import pandas as pd
train_data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Acoustic_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Acoustic_test_no_class.csv')

In [ ]:
train_data.head()

,Index,Class,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,...,_Chromagram_Mean_9,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy
0,1,sad,0.146,0.522,5.612,76.090,2.258,-0.947,0.697,-0.430,...,0.174,1.000,0.153,0.031,0.388,0.246,-0.030,2.243,0.751,0.969
1,2,happy,0.077,0.552,7.346,189.398,2.332,0.214,0.324,0.190,...,0.649,0.996,0.902,0.439,0.333,0.153,-0.002,1.346,0.818,0.971
2,3,relax,0.140,0.522,7.122,157.623,1.902,0.682,0.818,0.314,...,0.105,0.742,0.010,0.631,0.364,0.229,-0.162,2.691,0.758,0.967
3,4,relax,0.138,0.578,6.464,59.927,2.827,0.391,0.187,0.127,...,0.018,0.862,0.000,0.647,0.269,0.263,-0.039,0.498,0.748,0.964
4,5,happy,0.125,0.519,7.091,114.608,1.321,-0.336,0.529,-0.233,...,0.492,0.448,0.031,0.997,0.405,0.191,-0.019,1.922,0.818,0.969


In [ ]:
test_data.head()

,Index,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,_MFCC_Mean_5,...,_Chromagram_Mean_9,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy
0,1,0.101,0.582,6.039,110.470,1.932,-0.104,0.209,-0.173,0.336,...,0.101,0.017,0.000,1.000,0.259,0.218,0.014,0.374,0.678,0.962
1,2,0.131,0.552,10.261,151.685,1.708,0.729,0.831,0.334,0.258,...,0.024,0.005,0.061,0.016,0.325,0.189,0.055,1.495,0.751,0.967
2,3,0.100,0.548,5.388,164.528,3.062,0.901,0.978,0.469,0.247,...,0.741,0.001,0.473,0.870,0.285,0.245,0.041,0.673,0.598,0.973
3,4,0.115,0.568,6.193,133.742,4.521,-0.748,0.359,-0.205,0.073,...,0.002,1.000,0.355,0.004,0.371,0.272,0.032,2.243,0.690,0.966
4,5,0.135,0.473,10.157,112.205,1.832,0.760,0.691,0.669,0.358,...,0.387,1.000,0.141,0.009,0.380,0.206,-0.007,1.346,0.785,0.969


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score

**Preprocessing the training data**

In [ ]:
X_train = train_data.drop(columns=["Index", "Class"])
y_train = train_data["Class"]

**Scale feature**

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

**Split into train and validation sets**

In [ ]:
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=40
)

**Defining base model's**

In [ ]:
lr = LogisticRegression(max_iter=1000, random_state=40)
rf = RandomForestClassifier(n_estimators=100, random_state=40)
sv = SVC(probability=True, kernel='linear', random_state=40)

**Defining ensemble model using hard voting**

In [ ]:
ensemble = VotingClassifier(
    estimators=[
        ('logistic_reg', lr),
        ('random_forest', rf),
        ('svm', sv)
    ],
    voting='hard'
)

**Train the ensemble model**

In [ ]:
ensemble.fit(X_train_split, y_train_split)

VotingClassifier(estimators=[('logistic_reg',
                              LogisticRegression(max_iter=1000,
                                                 random_state=40)),
                             ('random_forest',
                              RandomForestClassifier(random_state=40)),
                             ('svm',
                              SVC(kernel='linear', probability=True,
                                  random_state=40))])

**Make predictions on the validation set**

In [ ]:
y_val_prediction = ensemble.predict(X_val)

**Evaluation of the model**

In [ ]:
print("Classification Report:")
print(classification_report(y_val, y_val_prediction))
print("Accuracy:", accuracy_score(y_val, y_val_prediction))

Classification Report:
              precision    recall  f1-score   support

       angry       0.91      0.87      0.89        23
       happy       0.65      0.92      0.76        12
       relax       1.00      0.75      0.86        12
         sad       0.67      0.62      0.64        13

    accuracy                           0.80        60
   macro avg       0.81      0.79      0.79        60
weighted avg       0.82      0.80      0.80        60

Accuracy: 0.8


**Preprocessing the test data**

In [ ]:
X_test = test_data.drop(columns=["Index"])
X_test_scaled = scaler.transform(X_test)

**Predictions on the test data**

In [ ]:
y_test_prediction = ensemble.predict(X_test_scaled)
print("Predicted Class Labels for Test Data:", y_test_prediction)

Predicted Class Labels for Test Data: ['sad' 'relax' 'relax' 'sad' 'relax' 'relax' 'angry' 'relax' 'sad' 'happy'
 'sad' 'angry' 'sad' 'relax' 'happy' 'sad' 'angry' 'angry' 'sad' 'angry'
 'relax' 'angry' 'angry' 'sad' 'sad' 'angry' 'happy' 'sad' 'angry' 'happy'
 'angry' 'happy' 'relax' 'relax' 'sad' 'angry' 'happy' 'angry' 'happy'
 'happy' 'angry' 'relax' 'relax' 'relax' 'sad' 'angry' 'happy' 'happy'
 'sad' 'happy' 'relax' 'angry' 'happy' 'relax' 'angry' 'sad' 'sad' 'happy'
 'relax' 'happy' 'happy' 'sad' 'happy' 'sad' 'happy' 'relax' 'angry' 'sad'
 'happy' 'relax' 'relax' 'happy' 'relax' 'relax' 'angry' 'sad' 'happy'
 'angry' 'happy' 'relax' 'relax' 'relax' 'sad' 'relax' 'sad' 'relax' 'sad'
 'angry' 'happy' 'relax' 'sad' 'angry' 'happy' 'sad' 'happy' 'angry'
 'angry' 'happy' 'happy' 'relax']


**Postprocessing of test data set**

In [ ]:
# Add the predicted class labels to the test data
test_data['Predicted_Class'] = y_test_prediction

# Save the test data with the predicted class labels to a CSV file
test_data.to_csv('Test_data_with_predictions.csv', index=False)